In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## Swing States

- Pennsylvania
- Michigan
- Wisconsin
- Georgia
- Arizona
- North Carolina
- Nevada

In [12]:
# Load the most recent 538 Data

df = pd.read_csv("Datasets/president_polls.csv")
pd.set_option("display.max_columns", None)
df.head()



,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,transparency_score,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,endorsed_candidate_id,endorsed_candidate_name,endorsed_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
0,85913,568,YouGov,NaN,NaN,YouGov,391,YouGov,B+,Online Panel,9.0,NaN,2/6/24,2/9/24,NaN,NaN,NaN,NaN,NaN,NaN,191992,869.0,rv,NaN,rv,NaN,2/9/24 15:34,NaN,https://d3nkl3psvxxpe9.cloudfront.net/document...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,44.0
1,85913,568,YouGov,NaN,NaN,YouGov,391,YouGov,B+,Online Panel,9.0,NaN,2/6/24,2/9/24,NaN,NaN,NaN,NaN,NaN,NaN,191992,869.0,rv,NaN,rv,NaN,2/9/24 15:34,NaN,https://d3nkl3psvxxpe9.cloudfront.net/document...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,45.0
2,85914,1015,Chism Strategies,NaN,NaN,Chism Strategies,470,Blueprint Polling,B/C,Text,1.0,Pennsylvania,2/6/24,2/8/24,NaN,NaN,NaN,NaN,NaN,NaN,191993,500.0,rv,NaN,rv,NaN,2/9/24 16:24,NaN,https://chismstrategies.com/would-a-taylor-swi...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,32.0
3,85914,1015,Chism Strategies,NaN,NaN,Chism Strategies,470,Blueprint Polling,B/C,Text,1.0,Pennsylvania,2/6/24,2/8/24,NaN,NaN,NaN,NaN,NaN,NaN,191993,500.0,rv,NaN,rv,NaN,2/9/24 16:24,NaN,https://chismstrategies.com/would-a-taylor-swi...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,40.0
4,85859,568,YouGov,352,Economist,YouGov,391,YouGov,B+,Online Panel,9.0,NaN,2/4/24,2/6/24,NaN,NaN,NaN,NaN,NaN,NaN,191853,1399.0,rv,NaN,rv,NaN,2/7/24 10:50,NaN,https://d3nkl3psvxxpe9.cloudfront.net/document...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,43.0


In [14]:
#Define a function to grab the data by state


def get_state_data(state,df):
    state_data = df[df['state'] == state]
    return state_data

pa = get_state_data('Pennsylvania',df)
mi = get_state_data('Michigan',df)
wi = get_state_data('Wisconsin',df)
ga = get_state_data('Georgia',df)
az = get_state_data('Arizona',df)
nc = get_state_data('North Carolina',df)
nv = get_state_data('Nevada',df)

In [17]:
def get_candidate_data(candidate,df):
    candidate_data = df[df['candidate_name'] == candidate]
    return candidate_data

biden = get_candidate_data('Joe Biden',pa)
biden.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,transparency_score,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,endorsed_candidate_id,endorsed_candidate_name,endorsed_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
2,85914,1015,Chism Strategies,NaN,NaN,Chism Strategies,470,Blueprint Polling,B/C,Text,1.0,Pennsylvania,2/6/24,2/8/24,NaN,NaN,NaN,NaN,NaN,NaN,191993,500.0,rv,NaN,rv,NaN,2/9/24 16:24,NaN,https://chismstrategies.com/would-a-taylor-swi...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,32.0
213,85797,1114,Franklin and Marshall College,NaN,NaN,Franklin & Marshall College Center for Opinion...,106,Franklin & Marshall College Center for Opinion...,B/C,Mail-to-Web/Mail-to-Phone,8.5,Pennsylvania,1/17/24,1/28/24,NaN,NaN,NaN,NaN,NaN,NaN,191558,507.0,rv,NaN,rv,NaN,2/1/24 12:25,NaN,https://www.fandmpoll.org/franklin-marshall-po...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,43.0
215,85797,1114,Franklin and Marshall College,NaN,NaN,Franklin & Marshall College Center for Opinion...,106,Franklin & Marshall College Center for Opinion...,B/C,Mail-to-Web/Mail-to-Phone,8.5,Pennsylvania,1/17/24,1/28/24,NaN,NaN,NaN,NaN,NaN,NaN,191559,499.0,rv,NaN,rv,NaN,2/1/24 12:25,NaN,https://www.fandmpoll.org/franklin-marshall-po...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,42.0
306,85770,1189,Morning Consult,96,Bloomberg,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,Pennsylvania,1/16/24,1/21/24,NaN,NaN,NaN,NaN,NaN,NaN,191330,795.0,rv,NaN,rv,NaN,1/31/24 10:44,NaN,https://pro-assets.morningconsult.com/wp-uploa...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,40.0
311,85770,1189,Morning Consult,96,Bloomberg,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,Pennsylvania,1/16/24,1/21/24,NaN,NaN,NaN,NaN,NaN,NaN,191331,795.0,rv,NaN,rv,NaN,1/31/24 10:44,NaN,https://pro-assets.morningconsult.com/wp-uploa...,NaN,NaN,NaN,8872,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,45.0
